# WEB SCRAPING DE LA PAGE LINKEDIN


## Le script se connecte à LinkedIn, recherche des posts en fonction de mots-clés, et extrait les données des posts

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os
import dateparser  

# Configurer Selenium WebDriver
driver_path = 'C:/Users/HP/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'  #chemin complet vers chromedriver.exe

# Vérification de l'existance du  fichier chromedriver 
if not os.path.exists(driver_path):
    raise FileNotFoundError(f"Le fichier chromedriver n'existe pas à l'emplacement : {driver_path}")

service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# URL de connexion LinkedIn
linkedin_login_url = 'https://www.linkedin.com/login'

# Identifiants LinkedIn
username = ''
password = ''

# Mots-clés de recherche
keywords = [
    "Cybersécurité en Afrique",
    "Sécurité informatique Afrique",
    "Cyber menaces Afrique",
    "Protection des données Afrique",
    "Intelligence artificielle Afrique",
    "IA en Afrique",
    "Technologies de sécurité Afrique",
    "Sécurité numérique Afrique",
    "Cyber résilience Afrique",
    "Gouvernance de la sécurité Afrique",
    "Risque cyber Afrique",
    "Cyber défense Afrique",
    "IA pour la sécurité Afrique",
    "Sécurité des réseaux Afrique",
    "Menaces numériques Afrique"
]

# Initialisation d'une liste pour stocker les données extraites
posts_data = []

# Fonction pour convertir les dates relatives en dates absolues
def convert_relative_date(relative_date):
    if relative_date:
        date = dateparser.parse(relative_date)
        if date:
            return date.strftime('%Y-%m-%d %H:%M:%S')
    return None

# Fonction pour se connecter à LinkedIn
def linkedin_login(driver, username, password):
    driver.get(linkedin_login_url)
    time.sleep(3)  # Attendre le chargement de la page

    email_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'username')))
    password_field = driver.find_element(By.ID, 'password')
    login_button = driver.find_element(By.XPATH, '//button[@type="submit"]')

    email_field.send_keys(username)
    password_field.send_keys(password)
    login_button.click()

    time.sleep(3)  # Attendre la connexion
    
#Fonctions pour verifier l'existence des selecteurs    

def is_class_exist(current_driver, _class_name_):
    try:
        current_driver.find_element(By.CLASS_NAME,_class_name_)
        return True
    except NoSuchElementException:
        return False
    
def is_element_exist(current_driver, _selector_name_):
    try:
        current_driver.find_element(By.CSS_SELECTOR,_selector_name_)
        return True
    except NoSuchElementException:
        return False
    
# Fonction pour scraper la page LinkedIn
def scrape_linkedin_page(driver, keyword):
    url = f"https://www.linkedin.com/search/results/content/?keywords={keyword}&origin=GLOBAL_SEARCH_HEADER"
    driver.get(url)
    time.sleep(5)  # Attendre le chargement de la page

    # Scroller pour charger plus de contenu
    body = driver.find_element(By.TAG_NAME, "body")
    for _ in range(20):  # nombre de scroll
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)

    # Trouver les éléments contenant les posts apres inspection de la page LinkedIn
    posts = driver.find_elements(By.CLASS_NAME, "artdeco-card")
    print(f"Nombre de posts trouvés pour '{keyword}': {len(posts)}")
    
    for post in posts:
        try:
            
            
            if is_class_exist(post,"update-components-update-v2__commentary"):
                content_elem = post.find_element(By.CLASS_NAME, "update-components-update-v2__commentary")              
            
            if is_class_exist(post,"update-components-actor__sub-description"):
                date_elem = post.find_element(By.CLASS_NAME, "update-components-actor__sub-description")
            
            if is_class_exist(post,"update-components-actor__name"):
                author_elem = post.find_element(By.CLASS_NAME, "update-components-actor__name") 
            
            if is_element_exist(post,".social-details-social-counts__reactions-count"):
                like_elem = post.find_element(By.CSS_SELECTOR, ".social-details-social-counts__reactions-count")            
            
            if is_element_exist(post,".social-details-social-counts__comments"):
                commentaire_elem = post.find_element(By.CSS_SELECTOR, ".social-details-social-counts__comments")
                
            if is_element_exist(post,"li.social-details-social-counts__comments + li"):
                repost_elem = post.find_element(By.CSS_SELECTOR, "li.social-details-social-counts__comments + li")    
                
            
            #repost_elem = post.find_element(By.CSS_SELECTOR, "li.social-details-social-counts__comments + li")
        
            
            if like_elem :
                like = like_elem.text.strip() if like_elem else 'N/A'
            else :
                like = 'N/A'
            
            if author_elem :
                author = author_elem.text.strip() if author_elem else 'N/A'
            else :
                author = 'N/A'
            
            if content_elem :
                content = content_elem.text.strip() if content_elem else 'N/A'
            else :
                content = 'N/A'
            
            if commentaire_elem and commentaire_elem.text.strip() != '' :
                 commentaire = commentaire_elem.text.strip()
            else:
                commentaire = 'N/A'
                
            if repost_elem :
                repost = repost_elem.text.strip() if repost_elem else 'N/A'
            else :
                repost = 'N/A'
                
            relative_date = date_elem.text.strip() if date_elem else 'N/A'

            # Convertir  date relative en date absolue
            date = convert_relative_date(relative_date)

            # Vérifier les données extraites
            print(f"Post: Like: {like}, Author: {author}, Date: {date}, Repost:{repost},commentaire = {commentaire}")

            posts_data.append({
                'Keyword': keyword,
                'Like': like,
                'Content': content,
                'Author': author,
                'Date': date,
                'Repost':repost,
                'Commentaire':commentaire
                
            })
        except Exception as e:
            print(f"Erreur lors de l'extraction d'un post : {e}")

# Se connecter à LinkedIn
linkedin_login(driver, username, password)

# Scraper les posts pour chaque mot-clé
for keyword in keywords:
    scrape_linkedin_page(driver, keyword)
    # Enregistrer les données partiellement après chaque mot-clé
    df_posts = pd.DataFrame(posts_data)
    df_posts.to_csv('linkedin_posts_partial.csv', index=False)

# Convertir les données en DataFrame
df_posts = pd.DataFrame(posts_data)

# Enregistrer les données dans un fichier CSV final
df_posts.to_csv('linkedin_posts.csv', index=False)

# Fermer le navigateur
driver.quit()

print("Données extraites et enregistrées dans linkedin_posts.csv")


Nombre de posts trouvés pour 'Cybersécurité en Afrique': 25
Erreur lors de l'extraction d'un post : cannot access local variable 'like_elem' where it is not associated with a value
Erreur lors de l'extraction d'un post : cannot access local variable 'commentaire_elem' where it is not associated with a value
Post: Like: 63, Author: data354
data354, Date: None, Repost:7 reposts,commentaire = 3 comments
Post: Like: 63, Author: Abidjanaises In Tech
Abidjanaises In Tech, Date: 2024-04-30 13:06:04, Repost:4 reposts,commentaire = 2 comments
Post: Like: 7, Author: Africa CyberSecurity Mag
Africa CyberSecurity Mag, Date: 2023-08-30 13:06:04, Repost:4 reposts,commentaire = 2 comments
Post: Like: 444, Author: Béranger GUEDOU
Béranger GUEDOU, Date: None, Repost:3 reposts,commentaire = 28 comments
Post: Like: 6, Author: Africa CyberSecurity Mag
Africa CyberSecurity Mag, Date: 2023-09-30 13:06:04, Repost:3 reposts,commentaire = 28 comments
Post: Like: 6, Author: data354
data354, Date: None, Repost:3

# Nettoyage et tansformations des données 

In [30]:
# Nettoyage des données extraites
import pandas as pd

# Charger les données extraites depuis le fichier CSV
df_posts = pd.read_csv('linkedin_posts_partial.csv')

# Inspection initiale des données
print("Aperçu des données:")
print(df_posts.head())

# Nettoyage des données
# Supprimer les doublons
df_posts.drop_duplicates(inplace=True)

# Gérer les valeurs manquantes
df_posts.fillna('N/A', inplace=True)

# Inspection après nettoyage
print("Données après nettoyage:")
print(df_posts.head())

# Stockage des données nettoyées dans un nouveau fichier CSV
df_posts.to_csv('linkedin_posts_cleaned.csv', index=False)

print("Données nettoyées et enregistrées dans linkedin_posts_cleaned.csv")

Aperçu des données:
                    Keyword  Like  \
0  Cybersécurité en Afrique    63   
1  Cybersécurité en Afrique    63   
2  Cybersécurité en Afrique     7   
3  Cybersécurité en Afrique   444   
4  Cybersécurité en Afrique     6   

                                             Content  \
0  Le Cyber Africa Forum (CAF) est le carrefour d...   
1  [ 💻 CYBER AFRICA FORUM J-3] En tant que membre...   
2  🔴Si vous n'êtes pas encore à jour sur les dern...   
3  Je quitte la France et voici pourquoi !\n\n\n\...   
4  🔴Si vous n'êtes pas à jour sur les dernières a...   

                                              Author                 Date  \
0                                   data354\ndata354                  NaN   
1         Abidjanaises In Tech\nAbidjanaises In Tech  2024-04-30 13:06:04   
2  Africa CyberSecurity Mag\nAfrica CyberSecurity...  2023-08-30 13:06:04   
3                   Béranger GUEDOU\nBéranger GUEDOU                  NaN   
4  Africa CyberSecurity Mag\nAfrica